Щоб запустити наступні ноутбуки, якщо ви ще цього не зробили, вам потрібно розгорнути модель, яка використовує `text-embedding-ada-002` як базову модель, і вказати ім'я розгортання у файлі .env як `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Далі ми завантажимо Індекс Вбудов у DataFrame Pandas. Індекс Вбудов зберігається у файлі JSON під назвою `embedding_index_3m.json`. Індекс Вбудов містить вбудовування для кожної з транскрипцій YouTube до кінця жовтня 2023 року.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Далі ми створимо функцію під назвою `get_videos`, яка шукатиме у Embedding Index за запитом. Функція повертатиме 5 відео, які найбільше схожі на запит. Ось як працює ця функція:

1. Спочатку створюється копія Embedding Index.
2. Далі для запиту обчислюється Embedding за допомогою OpenAI Embedding API.
3. Потім у Embedding Index створюється новий стовпець під назвою `similarity`. У стовпці `similarity` зберігається косинусна схожість між Embedding запиту та Embedding кожного відеосегмента.
4. Далі Embedding Index фільтрується за стовпцем `similarity`. У Embedding Index залишаються лише ті відео, у яких косинусна схожість більша або дорівнює 0.75.
5. Нарешті, Embedding Index сортується за стовпцем `similarity`, і повертаються 5 найкращих відео.


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Ця функція дуже проста, вона просто виводить результати пошукового запиту.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Спочатку індекс вбудовувань завантажується у DataFrame бібліотеки Pandas.
2. Далі користувачу пропонується ввести запит.
3. Потім викликається функція `get_videos`, щоб знайти запит в індексі вбудовувань.
4. Нарешті, викликається функція `display_results`, щоб показати результати користувачу.
5. Після цього користувачу знову пропонується ввести запит. Цей процес триває, поки користувач не введе `exit`.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.uk.png)

Вам буде запропоновано ввести запит. Введіть запит і натисніть Enter. Додаток поверне список відео, які відповідають вашому запиту. Також додаток надасть посилання на місце у відео, де знаходиться відповідь на ваше питання.

Ось декілька прикладів запитів, які можна спробувати:

- Що таке Azure Machine Learning?
- Як працюють згорткові нейронні мережі?
- Що таке нейронна мережа?
- Чи можу я використовувати Jupyter Notebooks з Azure Machine Learning?
- Що таке ONNX?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Відмова від відповідальності**:  
Цей документ було перекладено за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, звертаємо вашу увагу, що автоматичний переклад може містити помилки або неточності. Оригінальний документ рідною мовою слід вважати авторитетним джерелом. Для отримання критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильне тлумачення, що виникли внаслідок використання цього перекладу.
